<a href="https://colab.research.google.com/github/VictorFu0717/AI/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 2s 0us/step


In [ ]:
import glob
import os
dn = os.path.dirname(dataset)
fn = glob.glob(dn + "/aclImdb/train/pos/*")
with open(fn[0], "r", encoding="utf-8") as f:
    print("pos:", f.read())
fn = glob.glob(dn + "/aclImdb/train/neg/*")
with open(fn[0], "r", encoding="utf-8") as f:
    print("neg:", f.read())

pos: Tenshu is imprisoned and sentenced to death. When he survives electrocution the government officials give him a choice to either be electrocute at a greater degree or agree to some experiments. He chooses the experimentation and is placed in a large metallic cell with a bad ass criminal who also survived the electrocution. They can have whatever the want in the room (within reason), but they can't leave. after a few days there meals are cut down to one per day and the room temp is set up too 100. After some more alarms are sounded at intervals so they can't sleep. One day a 'witch' come into their cell (albeit a glassed off portion) What happens next I'll let you find out. I may be in the minority here but I liked the build up, it was intriguing to me. Now if the payoff was half as good as the build up was I would have rated this so much higher.<br /><br />My Grade: C+ <br /><br />Media Blaster's 2 DVD set Extras: Disc 1) Director's Cut; Trailers for "Versus", "Aragami", "Attack t

In [ ]:
import pandas as pd
def getdata(base):
    datas = {"article":[], "ans":[]}
    targets = os.path.join(base, "pos", "*")
    for fn in glob.glob(targets):
        with open(fn, "r", encoding="utf-8") as f:
            datas["article"].append(f.read())
            datas["ans"].append(1)
    targets = os.path.join(base, "neg", "*")
    for fn in glob.glob(targets):
        with open(fn, "r", encoding="utf-8") as f:
            datas["article"].append(f.read())
            datas["ans"].append(0)
    return pd.DataFrame(datas)
train = os.path.join(dn, "aclImdb", "train")
train_df = getdata(train)
test = os.path.join(dn, "aclImdb", "test")
test_df = getdata(test)

In [ ]:
test_df

,article,ans
0,Great British director Christopher Nolan (Mome...,1
1,"i hate vampire movies. with that said, this on...",1
2,Seldom is seen a film sequel that surpasses or...,1
3,I made a promise that if ever I posted a comme...,1
4,I never saw any of John Leguizamo's stand-up b...,1
...,...,...
24995,"I have loved the book ""A Little Princess"" for ...",0
24996,"PERHAPS SPOILER !! well, i ve seen it at the f...",0
24997,Two things -- too long and totally lacked cred...,0
24998,Great ensemble cast but unfortunately a bunch ...,0


In [ ]:
#num_words=3000最常出現的3000個單字才做處理，其他的忽略
#token將詞轉成數字 Tokenize: I -> 1 love -> 2 you -> 3
#texts to sequences將整句文字轉成數字列

from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["article"])
x_train_seq = tok.texts_to_sequences(train_df["article"])
x_test_seq = tok.texts_to_sequences(test_df["article"])

In [ ]:
# tok.word_index, 0不會用到, 0 for padding
tok.index_word[19]
# 有東西消失是因為超過三千最常出現單字
# sequences to texts將數字列轉成整句文字
tok.sequences_to_texts(x_train_seq)[0]

"is and to death when he the government give him a choice to either be at a greater degree or agree to some he the and is placed in a large cell with a bad ass criminal who also the they can have whatever the want in the room within reason but they can't leave after a few days there are cut down to one day and the room is set up too 100 after some more are sounded at so they can't sleep one day a come into their cell a off what happens next i'll let you find out i may be in the here but i liked the build up it was intriguing to me now if the was half as good as the build up was i would have rated this so much higher br br my grade c br br media 2 dvd set extras 1 director's cut for attack the gas station and deadly 2 theatrical cut commentary with cast and crew interview making of original trailer and"

In [ ]:
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,6,2,5,338,51,26,1,1365,199.0,87.0,3.0,1095.0,5.0,342.0,27.0,30.0,3.0,2781.0,2454.0,39.0,1037.0,5.0,46.0,26.0,1.0,2.0,6.0,2602.0,8.0,3.0,1053.0,2753.0,16.0,3.0,75.0,1992.0,1670.0,34.0,79.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191,587,215,392,16,39,32,37,132.0,39.0,148.0,10.0,37.0,95.0,196.0,56.0,1627.0,1.0,574.0,8.0,1299.0,747.0,38.0,120.0,525.0,649.0,1.0,179.0,882.0,6.0,9.0,976.0,5.0,18.0,364.0,16.0,1.0,639.0,1614.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,116,1,99,2,202,1,1,23.0,272.0,92.0,1.0,17.0,18.0,128.0,471.0,199.0,9.0,155.0,43.0,4.0,155.0,9.0,13.0,1187.0,44.0,1.0,99.0,185.0,1624.0,1187.0,10.0,59.0,25.0,38.0,2.0,10.0,241.0,665.0,329.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,83,55,10,216,11,17,10,1578.0,8.0,116.0,16.0,9.0,1.0,838.0,13.0,48.0,1056.0,58.0,688.0,83.0,2.0,10.0,870.0,3.0,596.0,18.0,302.0,185.0,5.0,103.0,3.0,542.0,1086.0,452.0,944.0,16.0,58.0,416.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,89,388,135,1,852,4,11,19.0,23.0,35.0,75.0,891.0,143.0,40.0,122.0,58.0,10.0,255.0,9.0,1189.0,527.0,2502.0,2.0,22.0,525.0,76.0,2735.0,80.0,1.0,327.0,4.0,3.0,1670.0,8.0,3.0,359.0,19.0,71.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,10,1922,12,11,19,45,620,1,4.0,3.0,173.0,4.0,9.0,239.0,46.0,4.0,95.0,18.0,9.0,283.0,12.0,75.0,70.0,634.0,27.0,1199.0,10.0,112.0,178.0,5.0,64.0,11.0,19.0,171.0,9.0,13.0,3.0,75.0,19.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,277,191,203,320,34,1578,122,1,390.0,4.0,1.0,700.0,627.0,53.0,8.0,3.0,389.0,510.0,16.0,3.0,436.0,16.0,1390.0,2.0,3.0,348.0,644.0,4.0,3.0,1539.0,16.0,46.0,1137.0,999.0,379.0,5.0,2172.0,53.0,1.0,510.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,1,678,8,1,422,45,50,1216,15.0,1.0,794.0,4.0,11.0,19.0,71.0,15.0,91.0,820.0,1481.0,30.0,219.0,33.0,89.0,25.0,5.0,1228.0,184.0,2.0,103.0,11.0,19.0,2101.0,91.0,202.0,7.0,7.0,141.0,5.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,721,704,11,321,20,371,8,13,875.0,5.0,76.0,140.0,42.0,185.0,1.0,797.0,596.0,12.0,1388.0,532.0,8.0,3.0,862.0,17.0,18.0,598.0,4.0,1.0,2595.0,838.0,5.0,94.0,9.0,287.0,134.0,7.0,7.0,801.0,676.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#每篇文章長度不一樣，截長補短讓長度一樣 (訓練的時候無法處理長短不同的文章)
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=1024)
x_test_pad = pad_sequences(x_test_seq, maxlen=1024)

In [ ]:
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,11,35,73,1927,7,7,58,1239,1144,7,7,1773,238,285,267,2253,297,2024,602,15,1271,1,2547,1706,2,2485,238,2246,602,1700,16,174,2,1048,2696,228,4,201,1467,2
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,51,56,5,1,1211,20,5,1600,1700,20,921,2,1166,2205,1829,11,28,6,542,1174,18,61,15,145,34,128,25,3,160,2,158,1581,9,8,65,88,1132,2196,39,289
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,39,124,21,37,369,39,22,121,40,1339,2137,44,10,97,1577,20,58,86,8,116,2,86,108,208,10,25,329,1,10,59,518,43,4,15,249,18,1395,47,6,207
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,121,12,440,862,304,10,25,309,1906,15,24,190,20,11,353,11,17,6,84,15,98,55,22,25,3,15,3,18,42,1,400,1574,1957,22,401,9,15,32,29,17
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,11,28,10,12,11,19,77,27,3,663,566,20,285,277,81,64,9,15,529,2,568,2640,688,5,1,1618,1983,273,43,199,9,3,321,22,235,27,1260,22,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2030,808,108,136,119,21,1183,742,1,2003,4,1,111,15,460,1,133,163,114,278,8,1,62,128,9,283,1,246,1723,19,123,90,6,9,8,1,347,155,246,195
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,91,667,7,7,10,188,63,383,1,19,91,287,3,321,44,332,3,334,4,1,153,39,23,3,663,334,4,1723,435,8,29,91,18,895,11,6,40,3,1386
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7,8,3,678,89,434,126,55,16,11,28,58,319,2,10,654,72,158,18,30,219,72,1072,260,202,31,8,995,1367,8,1,19,37,9,13,733,1063,748,2080,8
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,303,5,27,296,448,4,1,34,188,1195,723,33,178,5,294,9,727,39,468,9,80,3,209,28,106,3,425,1121,4,262,6,2272,31,3,325,230,681,34,2464,16


In [ ]:
#Embedding詞嵌入((3000詞＋1padding),128種情緒）
#mask_zero：True, 0不列入運算，視為padding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding
layers = [
    Embedding(3001, 128, mask_zero=True, input_length=1024),
    Flatten(),
    Dense(256, activation="relu"),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1024, 128)         384128    
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               33554688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 33,939,330
Trainable params: 33,939,330
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 不需要自己做one-hot
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
import numpy as np
y_train = np.array(train_df["ans"])
y_test = np.array(test_df["ans"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
   EarlyStopping(patience=5, restore_best_weights=True),
   ModelCheckpoint("sentiment.h5", save_best_only=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=200,
     epochs=50,
     validation_split=0.1,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 10s - loss: 0.7934 - accuracy: 0.6935 - val_loss: 0.2730 - val_accuracy: 0.8880
Epoch 2/50
113/113 - 7s - loss: 0.2156 - accuracy: 0.9186 - val_loss: 0.3496 - val_accuracy: 0.8404
Epoch 3/50
113/113 - 7s - loss: 0.0886 - accuracy: 0.9786 - val_loss: 0.4390 - val_accuracy: 0.8196
Epoch 4/50
113/113 - 7s - loss: 0.0270 - accuracy: 0.9976 - val_loss: 0.4990 - val_accuracy: 0.8216
Epoch 5/50
113/113 - 7s - loss: 0.0089 - accuracy: 0.9997 - val_loss: 0.5263 - val_accuracy: 0.8332
Epoch 6/50
113/113 - 7s - loss: 0.0035 - accuracy: 0.9998 - val_loss: 0.3839 - val_accuracy: 0.8848


In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.3201 - accuracy: 0.8629


[0.3201254606246948, 0.8628799915313721]